# CLIP-guided Diffusion — Lightweight Prototypes

Objective: prototype text-driven image generation and style-transfer using CLIP guidance to inform early product demos.

**Contents**
1. Environment & safety note
2. Text-to-image prototype
3. Image-to-image (style transfer) prototype
4. Observations & limitations


In [ ]:
# %% Environment (comment out installs if pre-installed)
# !pip install --quiet diffusers transformers accelerate torch safetensors pillow
from dataclasses import dataclass
from typing import Optional
import torch
from PIL import Image
import io

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

### Safety & licensing
- Use models in accordance with their licenses and terms.
- Add a basic prompt/NSFW filter when exposing a public UI.


In [ ]:
# %% Text-to-image (Stable Diffusion v1.5 example)
from diffusers import StableDiffusionPipeline

def generate(prompt: str, steps: int = 30, guidance: float = 7.5, seed: Optional[int] = 1234):
    pipe = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5')
    pipe = pipe.to(DEVICE)
    g = torch.Generator(device=DEVICE).manual_seed(seed) if seed is not None else None
    out = pipe(prompt, num_inference_steps=steps, guidance_scale=guidance, generator=g)
    return out.images[0]

# Example (skip execution in CI):
DO_RUN = False
if DO_RUN:
    img = generate('product mockup, clean studio lighting, minimal background')
    display(img)

In [ ]:
# %% Image-to-image (style nudging)
from diffusers import StableDiffusionImg2ImgPipeline

def stylize(init_img: Image.Image, prompt: str, strength: float = 0.6, steps: int = 30, seed: Optional[int] = 1234):
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained('runwayml/stable-diffusion-v1-5')
    pipe = pipe.to(DEVICE)
    g = torch.Generator(device=DEVICE).manual_seed(seed) if seed is not None else None
    out = pipe(prompt=prompt, image=init_img, strength=strength, num_inference_steps=steps, generator=g)
    return out.images[0]

# Example (skip execution):
DO_RUN = False
if DO_RUN:
    base = Image.new('RGB', (512,512), 'white')
    out = stylize(base, 'isometric icon style, soft gradients, crisp edges')
    display(out)

## Observations
- For quick demos, a single A10G/T4 is adequate; batch size 1, 512x512.
- Prompt engineering matters more than step counts for early concept checks.
- For consistent branding, keep a small prompt library checked into the repo.

## Next steps
- Wire a basic Streamlit UI to collect prompts and seed; export PNGs.
- If going beyond prototype, consider ControlNet or fine-tuning for product styles.
